## APPLIED DATA SCIENCE CAPSTONE PROJECT - WHERE TO OPEN A COFFEE SHOP IN IOWA CITY, IA


### Import Packages

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
import numpy as np
import os

In [3]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Import Dataframe

In [6]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Latitutde,Longitude
0,Bluffwood,41.671768,-91.496234
1,College Green,41.661205,-91.522863
2,Creekside,41.651514,-91.508291
3,Goosetown,41.666876,-91.520077
4,Longfellow,41.653495,-91.518021


In [7]:
 df.columns = ['Neighborhood', 'Latitude', 'Longitude']
df.head()

,Neighborhood,Latitude,Longitude
0,Bluffwood,41.671768,-91.496234
1,College Green,41.661205,-91.522863
2,Creekside,41.651514,-91.508291
3,Goosetown,41.666876,-91.520077
4,Longfellow,41.653495,-91.518021


### Use geopy library to get the latitude and longitude values of Iowa City

In [8]:
address = 'Iowa City, IA, USA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Iowa City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Iowa City are 41.6612561, -91.5299106.


### Make a map of Iowa City and put neighborhoods on top

In [9]:
# create map of Iowa City using latitude and longitude values
map_IC = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, Neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_IC)  
    
map_IC

### Define Foursquare credentials

In [11]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: SECRET
CLIENT_SECRET:SECRET


### Create function to explore all neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run function on all neighborhoods

In [13]:
IowaCity_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bluffwood
College Green
Creekside
Goosetown
Longfellow
Lucas Farms
Manville Heights
Melrose Ave
Miller Orchard
Morningside/Glendale
Parkview Terrace/Normandy
Northside
Oak Grove
Peninsula Area Neighborhood
Washington Hills
Windsor Ridge
Eastside
Shimek
Wetherby
Grantwood
South Pointe
 Pepperwood
Broadway


### Create dataframe using onehot encoding

In [14]:
# one hot encoding
IowaCity_onehot = pd.get_dummies(IowaCity_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
IowaCity_onehot['Neighborhood'] = IowaCity_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [IowaCity_onehot.columns[-1]] + list(IowaCity_onehot.columns[:-1])
IowaCity_onehot = IowaCity_onehot[fixed_columns]

IowaCity_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bar,Basketball Court,Beer Garden,...,Scenic Lookout,Shipping Store,Smoke Shop,Social Club,Sporting Goods Shop,Thrift / Vintage Store,Trail,Video Store,Winery,Zoo
0,College Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,College Green,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,College Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,College Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,College Green,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
IowaCity_onehot

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bar,Basketball Court,Beer Garden,Bookstore,Breakfast Spot,Burrito Place,Clothing Store,Coffee Shop,Diner,Discount Store,Dive Bar,Event Space,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Garden Center,Gas Station,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Home Service,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Laundromat,Lounge,Massage Studio,Mexican Restaurant,Miscellaneous Shop,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Record Shop,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Smoke Shop,Social Club,Sporting Goods Shop,Thrift / Vintage Store,Trail,Video Store,Winery,Zoo
0,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,College Green,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Creekside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Creekside,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Creekside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Creekside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9,Creekside,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Create dataframe grouped by neighborhood with mean for each venue


In [17]:
IowaCity_grouped = IowaCity_onehot.groupby('Neighborhood').mean().reset_index()
IowaCity_grouped

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bar,Basketball Court,Beer Garden,Bookstore,Breakfast Spot,Burrito Place,Clothing Store,Coffee Shop,Diner,Discount Store,Dive Bar,Event Space,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Garden Center,Gas Station,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Home Service,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Laundromat,Lounge,Massage Studio,Mexican Restaurant,Miscellaneous Shop,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Record Shop,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Smoke Shop,Social Club,Sporting Goods Shop,Thrift / Vintage Store,Trail,Video Store,Winery,Zoo
0,Pepperwood,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0000,0.333333,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
1,Broadway,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.111111,0.000000,0.000000,0.000000,0.0000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.111111,0.000000,0.000000,0.111111,0.0,0.000000,0.111111,0.0,0.000000,0.0000,0.000000
2,College Green,0.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.2000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
3,Creekside,0.0,0.000000,0.0,0.142857,0.000000,0.000000,0.142857,0.0,0.0000,0.0000,0.000000,0.000000,0.142857,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.142857,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.142857,0.0000,0.000000
4,Eastside,0.0,0.000000,0.0,0.000000,0.000000,0.200000,0.200000,0.0,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
5,Goosetown,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
6,Grantwood,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,

### Create dataframe grouped by neighborhood with sum for each venue

In [18]:
IowaCity_grouped1 = IowaCity_onehot.groupby('Neighborhood').sum().reset_index()
IowaCity_grouped1

,Neighborhood,American Restaurant,Antique Shop,Art Gallery,Athletics & Sports,Automotive Shop,Bakery,Bar,Basketball Court,Beer Garden,Bookstore,Breakfast Spot,Burrito Place,Clothing Store,Coffee Shop,Diner,Discount Store,Dive Bar,Event Space,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Garden Center,Gas Station,Grocery Store,Gym,Gym / Fitness Center,Historic Site,Home Service,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Laundromat,Lounge,Massage Studio,Mexican Restaurant,Miscellaneous Shop,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Record Shop,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Smoke Shop,Social Club,Sporting Goods Shop,Thrift / Vintage Store,Trail,Video Store,Winery,Zoo
0,Pepperwood,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Broadway,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0
2,College Green,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Creekside,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Eastside,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Goosetown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,Grantwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,Longfellow,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Manville Heights,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Melrose Ave,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


### Create a dataframe with just neighborhoods and coffee shop venue totals

In [19]:
IowaCity_grouped1[['Neighborhood','Coffee Shop']]

,Neighborhood,Coffee Shop
0,Pepperwood,0
1,Broadway,0
2,College Green,0
3,Creekside,0
4,Eastside,0
5,Goosetown,0
6,Grantwood,0
7,Longfellow,0
8,Manville Heights,0
9,Melrose Ave,0


### Create a dataframe of venues only serving food and beverages

In [20]:
IowaCity1 = IowaCity_grouped1[['Neighborhood','Bakery','Bar', 'Beer Garden', 'Breakfast Spot', 'Burrito Place', 'Coffee Shop', 'Diner', 'Dive Bar', 'Falafel Restaurant', 'Fast Food Restaurant', 'Fried Chicken Joint', 'Ice Cream Shop', 'Italian Restaurant', 'Japanese Restaurant', 'Juice Bar', 'Mexican Restaurant', 'Pizza Place', 'Pub', 'Restaurant', 'Sandwich Place']]

In [21]:
IowaCity1

,Neighborhood,Bakery,Bar,Beer Garden,Breakfast Spot,Burrito Place,Coffee Shop,Diner,Dive Bar,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place
0,Pepperwood,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Broadway,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,1
2,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Creekside,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,Eastside,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Goosetown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,Grantwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Longfellow,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Manville Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Melrose Ave,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


### Add new column called 'sum' to total all food/beverage service venues in neighborhood

In [22]:
IowaCity1['Sum'] = IowaCity1.sum(axis=1)
IowaCity1

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhood,Bakery,Bar,Beer Garden,Breakfast Spot,Burrito Place,Coffee Shop,Diner,Dive Bar,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Juice Bar,Mexican Restaurant,Pizza Place,Pub,Restaurant,Sandwich Place,Sum
0,Pepperwood,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2
1,Broadway,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,1,5
2,College Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,Creekside,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,3
4,Eastside,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
5,Goosetown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
6,Grantwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Longfellow,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,Manville Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Melrose Ave,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,2


### Create dataframe with just neighborhood and sum for report

In [23]:
IowaCity1[['Neighborhood', 'Sum']]

,Neighborhood,Sum
0,Pepperwood,2
1,Broadway,5
2,College Green,1
3,Creekside,3
4,Eastside,2
5,Goosetown,1
6,Grantwood,0
7,Longfellow,1
8,Manville Heights,0
9,Melrose Ave,2


### List top 5 venues in every neighborhood

In [24]:
num_top_venues = 5

for hood in IowaCity_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = IowaCity_grouped[IowaCity_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Pepperwood----
                venue  freq
0  Italian Restaurant  0.33
1      Breakfast Spot  0.33
2                Park  0.33
3         Record Shop  0.00
4                Lake  0.00


----Broadway----
                venue  freq
0         Pizza Place  0.22
1          Smoke Shop  0.11
2  Mexican Restaurant  0.11
3      Sandwich Place  0.11
4                 Gym  0.11


----College Green----
                 venue  freq
0  American Restaurant   0.2
1        Historic Site   0.2
2        Grocery Store   0.2
3                River   0.2
4          Pizza Place   0.2


----Creekside----
                venue  freq
0         Pizza Place  0.14
1  Athletics & Sports  0.14
2         Video Store  0.14
3            Pharmacy  0.14
4                 Bar  0.14


----Eastside----
          venue  freq
0  Home Service   0.2
1  Optical Shop   0.2
2        Bakery   0.2
3           Bar   0.2
4   Record Shop   0.2


----Goosetown----
                 venue  freq
0                 Park   0.5
1       Sa

### Create dataframe with top venues for each neighborhood

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = IowaCity_grouped['Neighborhood']

for ind in np.arange(IowaCity_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(IowaCity_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pepperwood,Italian Restaurant,Breakfast Spot,Park,Dive Bar,Hotel,Home Service,Historic Site,Gym / Fitness Center,Gym,Grocery Store
1,Broadway,Pizza Place,Fast Food Restaurant,Gym,Thrift / Vintage Store,Miscellaneous Shop,Smoke Shop,Mexican Restaurant,Sandwich Place,Dive Bar,Event Space
2,College Green,American Restaurant,Pizza Place,Historic Site,Grocery Store,River,Home Service,Gym / Fitness Center,Gym,Gas Station,Garden Center
3,Creekside,Video Store,Pharmacy,Pizza Place,Athletics & Sports,Ice Cream Shop,Bar,Clothing Store,Zoo,Fried Chicken Joint,Falafel Restaurant
4,Eastside,Home Service,Record Shop,Bakery,Bar,Optical Shop,Zoo,Garden Center,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint
5,Goosetown,Sandwich Place,Park,Zoo,Discount Store,Home Service,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Gas Station
6,Grantwood,Park,Scenic Lookout,Zoo,Discount Store,Home Service,Historic Site,Gym / Fitness Center,Gym,Grocery Store,Gas Station
7,Longfellow,Antique Shop,Historic Site,Bakery,Zoo,Dive Bar,Hotel,Home Service,Gym / Fitness Center,Gym,Grocery Store
8,Manville Heights,Antique Shop,Park,Zoo,Dive Bar,Hotel,Home Service,Historic Site,Gym / Fitness Center,Gym,Grocery Store
9,Melrose Ave,Basketball Court,Sandwich Place,Video Store,Ice Cream Shop,Gas Station,Event Space,Falafel Restaurant,Fast Food Restaurant,Fried Chicken Joint,Garden Center
